In [19]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import davies_bouldin_score
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
customers_df = pd.read_csv('Customers.csv')
products_df = pd.read_csv('Products.csv')
transactions_df = pd.read_csv('Transactions.csv')

In [21]:
customers_df.isnull().sum(), products_df.isnull().sum(), transactions_df.isnull().sum()

(CustomerID      0
 CustomerName    0
 Region          0
 SignupDate      0
 dtype: int64,
 ProductID      0
 ProductName    0
 Category       0
 Price          0
 dtype: int64,
 TransactionID      0
 CustomerID         0
 ProductID          0
 TransactionDate    0
 Quantity           0
 TotalValue         0
 Price              0
 dtype: int64)

In [22]:
customers_df = customers_df.drop_duplicates()
transactions_df = transactions_df.drop_duplicates()

In [23]:
customers_df.columns = customers_df.columns.str.strip().str.lower()
customers_df.columns

Index(['customerid', 'customername', 'region', 'signupdate'], dtype='object')

In [24]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Sample Customer and Transaction Data (replace with your actual dataset)
customers_df = pd.DataFrame({
    'CustomerID': ['C001', 'C002', 'C003', 'C004', 'C005'],
    'Age': [25, 34, 28, 40, 23],
    'AnnualIncome': [50000, 70000, 65000, 80000, 45000],
    'TransactionCount': [5, 8, 4, 10, 3],
    'AvgTransactionValue': [150, 200, 120, 180, 100],
})

transactions_df = pd.DataFrame({
    'TransactionID': ['T001', 'T002', 'T003', 'T004', 'T005'],
    'CustomerID': ['C001', 'C002', 'C003', 'C004', 'C005'],
    'ProductID': ['P001', 'P002', 'P003', 'P004', 'P005'],
    'TotalValue': [150, 200, 120, 180, 100],
})

# Merging customer data with transaction data to create a profile
customer_transaction_data = pd.merge(customers_df, transactions_df.groupby('CustomerID')['TotalValue'].sum().reset_index(), on='CustomerID')
customer_transaction_data['TotalValue'] = customer_transaction_data['TotalValue'].fillna(0)

# Function to get lookalikes based on cosine similarity
def get_lookalikes(customer_id, top_n=3):
    # Extract features for similarity computation
    customer_profile = customer_transaction_data[customer_transaction_data['CustomerID'] == customer_id]
    other_customers = customer_transaction_data[customer_transaction_data['CustomerID'] != customer_id]

    # Compute similarity using selected features (e.g., Age, AnnualIncome, TransactionCount, TotalValue)
    features = ['Age', 'AnnualIncome', 'TransactionCount', 'TotalValue']
    customer_features = customer_profile[features].values
    other_customers_features = other_customers[features].values

    # Calculate cosine similarity
    similarity_scores = cosine_similarity(customer_features, other_customers_features).flatten()

    # Get the top N most similar customers
    similar_customer_ids = other_customers['CustomerID'].iloc[np.argsort(similarity_scores)[::-1][:top_n]]
    similar_scores = np.sort(similarity_scores)[::-1][:top_n]

    # Return the results as a dictionary
    return list(zip(similar_customer_ids, similar_scores))

# Get top 3 lookalikes for Customer C001
lookalikes = get_lookalikes('C001', top_n=3)
print("Top 3 Lookalikes for Customer C001:")
for idx, (customer, score) in enumerate(lookalikes, 1):
    print(f"{idx}. Customer ID: {customer}, Similarity Score: {score}")


Top 3 Lookalikes for Customer C001:
1. Customer ID: C002, Similarity Score: 0.9999999895920186
2. Customer ID: C004, Similarity Score: 0.9999997184414862
3. Customer ID: C005, Similarity Score: 0.9999996969178286
